<a href="https://colab.research.google.com/github/davidjcox333/Man-vs-Machine/blob/master/GME_fits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the environment

In [ ]:
# Set working directory
from google.colab import drive, files
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/Multivariate Discounting/Scripts and Notebooks/'

Mounted at /content/gdrive
/content/gdrive/My Drive/Multivariate Discounting/Scripts and Notebooks


In [ ]:
# Connect TPU, if desired
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.7.0
Running on TPU  ['10.6.70.234:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.6.70.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.6.70.234:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
# System
import os
import sys
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

# Data Manipulation
import pandas as pd
import numpy as np

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# Analytics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from math import log
import scipy.stats as stats
from sklearn import metrics
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import PolynomialFeatures
import random

# Script preference
pd.set_option("display.max_columns", None)

# Make sure dataframe is ready for fitting

In [ ]:
# Read in the data
df = pd.read_csv('../Data/01_raw/data_long.csv').drop(['Unnamed: 0'], axis=1)

#%% One hot encode categorical variables for predictions
encode_df = pd.DataFrame(df['indiff_point'])
encode_df.columns = ['indiff_point']
encode_df = pd.concat([encode_df, pd.get_dummies(df['Group'], prefix='Group')], axis=1)
encode_df = pd.concat([encode_df, pd.get_dummies(df['Commodity'], prefix='Commodity')], axis=1)
encode_df = pd.concat([encode_df, pd.get_dummies(df['Gain_Loss'], prefix='Gain_Loss')], axis=1)
encode_df = pd.concat([encode_df, pd.get_dummies(df['Delay_Prob'], prefix='Delay_Prob')], axis=1)
encode_df['Amount'] = df['Amount']
encode_df['IV'] = df['IV']
encode_df['pid'] = df['pid']
encode_df = encode_df.dropna()
encode_df = encode_df.reset_index(drop=True)

encode_df = encode_df.drop(['Group_Control', 'Gain_Loss_Loss', 'Delay_Prob_Probability'], axis=1)

encode_df.head(20)

,indiff_point,Group_Cocaine,Commodity_Cocaine,Commodity_Health,Commodity_Money,Gain_Loss_Gains,Delay_Prob_Delay,Amount,IV,pid
0,0.75,1,1,0,0,1,1,100,1.0000,GM33
1,0.63,1,1,0,0,1,1,100,7.0000,GM33
2,0.61,1,1,0,0,1,1,100,30.4375,GM33
3,0.50,1,1,0,0,1,1,100,182.6250,GM33
4,0.35,1,1,0,0,1,1,100,365.0000,GM33
5,0.24,1,1,0,0,1,1,100,1825.0000,GM33
6,0.20,1,1,0,0,1,1,100,9125.0000,GM33
7,0.75,1,1,0,0,1,1,1000,1.0000,GM33
8,0.75,1,1,0,0,1,1,1000,7.0000,GM33
9,0.75,1,1,0,0,1,1,1000,30.4375,GM33


# Functions

In [ ]:
def calc_aic(n, loss, num_params):
  aic = n * log(loss) + 2 * num_params
  return aic

def calc_bic(n, loss, num_params):
  bic = n * log(loss) + num_params * log(n)
  return bic

def plot_pred(predictors, observed_data, title, save_name=None, 
              rand_state=5687463, model=None, plot_cond=False):
  # Fit the model
  if model=='RF':
      regr = RFR(max_depth=5, random_state=rand_state)
      regr.fit(predictors, observed_data)
  elif model=='LR':
      regr = LinearRegression()
      regr.fit(predictors, observed_data)
  elif model=='GBR':
      regr = GradientBoostingRegressor(random_state=rand_state)
      regr.fit(predictors, observed_data)
  elif model=='ABR':
      regr = AdaBoostRegressor(random_state=rand_state)
      regr.fit(predictors, observed_data)
  elif model == 'BR':
      regr = linear_model.BayesianRidge()
      regr.fit(predictors, observed_data)
  elif model == 'LaR':
      regr = linear_model.Lasso(alpha=0.1)  
  regr.fit(predictors, observed_data)
  # Get loss metrics
  predictions = regr.predict(predictors)
  try:
    num_params = len(regr.coef_) + 1
  except:
    num_params = np.nan
  r2 = metrics.r2_score(observed_data, predictions)
  mse = metrics.mean_squared_error(observed_data, predictions)
  rmse = np.sqrt(mse)
  mae = metrics.mean_absolute_error(observed_data, predictions)
  try:
    aic = calc_aic(n=len(observed_data), loss=mse, num_params=num_params)
  except:
    aic = np.nan
  try:
    bic = calc_bic(n=len(observed_data), loss=mse, num_params=num_params)
  except:
    bic= np.nan
  
  # Plot if desired
  if plot_cond==True:
    # Plot it
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.regplot(x=observed_data, y=regr.predict(predictors), color='k', )
    plt.title(title, fontsize=36, pad=16)
    ax.text(0.7, 0.08, r"$r^2=$"+f'{round(r2, 2)}', fontsize=24)
    plt.xticks(fontsize=20)
    plt.xlim(0, 1)
    plt.yticks(fontsize=20)
    plt.ylim(0, 1)
    plt.ylabel("Predicted Indifference Proportion", fontsize=30, labelpad=20)
    plt.xlabel("Observed Indifference Proportion", fontsize=30, labelpad=20)
    plt.savefig(f'../Figures/{save_name}.png', bbox_inches="tight")
    plt.show()

  return r2, mse, rmse, mae, aic, bic

## Model types to fit: 

1. Linear Regression (LR)
2. Mixed-Effects Model (ME)
3. Random Forest (RF)
4. Gradient Boosting (GBR)
5. Adaptive Boosting (ABR)
6. Bayesian Regression (BR)
7. Lasso Regression (LaR)



### WARNING: The following code block takes a long time to run. Break into batches if needed. 

In [ ]:
# Specify model
model = 'LR'

# Lists to store vals for later
fit = []
vac = []
mse =[]
mae = []
rmse =[]
aic = []
bic = []
models = []
group = []

# Predictor cols
preds = ['Group_Cocaine', 'Commodity_Cocaine', 'Commodity_Health', 'Commodity_Money',
          'Gain_Loss_Gains', 'Delay_Prob_Delay', 'Amount', 'IV']

for i in range(0, 100):
    if i%25==0:
      plot_cond=True
    else:
      plot_cond=False
    rand_state = int(round(random.random()*10000, 0))
    trim_df = encode_df.sample(n=int(round((len(encode_df)*0.8), 2)), replace=True, random_state=rand_state)
    # Fit to all data
    fit.append('all_'+model)
    vac_, mse_, mae_, rmse_, aic_, bic_ = plot_pred(predictors=trim_df[preds].astype(float),
                                        observed_data=trim_df[['indiff_point']].astype(float).to_numpy(),
                                        title=f'{model}: All Participants', 
                                        save_name=f'reg_plot_all_participants_biter_{model}_{i}',
                                        rand_state=rand_state, 
                                        model=model, plot_cond=plot_cond)
    vac.append(vac_)
    mae.append(mae_)
    mse.append(mse_)
    rmse.append(rmse_)
    aic.append(aic_)
    bic.append(bic_)
    group.append('all')
    models.append(model)
    
    # Cocaine group only
    fit.append('cocaine_'+model)
    temp_df = trim_df[trim_df['Group_Cocaine']==1]
    vac_, mse_, mae_, rmse_, aic_, bic_ = plot_pred(predictors=temp_df[preds].astype(float),
                                        observed_data=temp_df[['indiff_point']].astype(float).to_numpy(),
                                        title=f'{model}: All Participants', save_name=f'reg_plot_cocaine_biter_{model}_{i}',
                                        rand_state=rand_state, model=model, plot_cond=plot_cond)
    vac.append(vac_)
    mae.append(mae_)
    mse.append(mse_)
    rmse.append(rmse_)
    aic.append(aic_)
    bic.append(bic_)
    group.append('cocaine')
    models.append(model)
    
    # Control group only
    fit.append('control_'+model)
    temp_df = trim_df[trim_df['Group_Cocaine']==0]
    vac_, mse_, mae_, rmse_, aic_, bic_ = plot_pred(predictors=temp_df[preds].astype(float),
                                        observed_data=temp_df[['indiff_point']].astype(float).to_numpy(),
                                        title=f'{model}: All Participants', save_name=f'reg_plot_control_biter_{model}_{i}',
                                        rand_state=rand_state, model=model, plot_cond=plot_cond)
    vac.append(vac_)
    mae.append(mae_)
    mse.append(mse_)
    rmse.append(rmse_)
    aic.append(aic_)
    bic.append(bic_)
    group.append('control')
    models.append(model)
    
    # Fit to individual participant data
    for p in encode_df['pid'].unique():
        if random.random()>0.99:
          plot_cond=True
        else:
          plot_cond=False
        fit.append('ind_'+model)
        temp_df = trim_df[trim_df['pid']==p].dropna()
        temp_df = temp_df.reset_index(drop=True)
        vac_, mse_, mae_, rmse_, aic_, bic_ = plot_pred(predictors=temp_df[['Group_Cocaine', 'Commodity_Cocaine', 'Commodity_Health', 'Commodity_Money', 'Gain_Loss_Gains', 'Delay_Prob_Delay', 'Amount', 'IV']].astype(float),
                                            observed_data=temp_df[['indiff_point']].astype(float).to_numpy(),
                                            title=f'{model}: Participant {p}', save_name=f'reg_plot{model}_{p}_biter_{i}',
                                            rand_state=rand_state, model=model, plot_cond=plot_cond)
        vac.append(vac_)
        mae.append(mae_)
        mse.append(mse_)
        rmse.append(rmse_)
        aic.append(aic_)
        bic.append(bic_)
        if temp_df['Group_Cocaine'][0]==1:
          group.append('cocaine')
        else:
          group.append('control')
        models.append(model)

    # Update notebook user
    clear_output()
    print(f'Using {model}, {(i+1)} iterations complete ({round(100*((i+1)/100), 2)}%)')

    # Ongoing saves
    if i%10==0:
      # Make a df
      fit_df = pd.DataFrame({'fit':fit, 'vac':vac, 'mse':mse, 'rmse':rmse, 
                             'mae':mae, 'aic':aic, 'bic':bic, 'model': models, 'group':group})
      fit_df.to_csv(f'../Data/02_initial_models_output/boostrapped_ml_regressors_{model}.csv')

# Final save
fit_df = pd.DataFrame({'fit':fit, 'vac':vac, 'mse':mse, 'rmse':rmse, 
                       'mae':mae, 'aic':aic, 'bic':bic, 'model': models, 'group':group})
fit_df.to_csv(f'../Data/02_initial_models_output/boostrapped_ml_regressors_{model}.csv')

Using LR, 100 iterations complete (100.0%)


### Fitting linear mixed-effects models which require a bit of a different approach

In [ ]:
# Using bootstrapped samples to fit MLM to all data
model = 'ME'
import warnings
warnings.filterwarnings("ignore")
fit = []
vac = []
mse =[]
mae = []
rmse =[]
aic = []
bic = []
models = []
group = []

# Predictor cols
preds = ['Group_Cocaine', 'Commodity_Cocaine', 'Commodity_Health', 'Commodity_Money',
          'Gain_Loss_Gains', 'Delay_Prob_Delay', 'Amount', 'IV']

for i in range(0, 100):
  rand_state = int(round(random.random()*10000, 0))
  trim_df = encode_df.sample(n=int(round((len(encode_df)*0.8), 2)), replace=True, random_state=rand_state)
  trim_df = trim_df.reset_index(drop=True)
  # Fit model
  regr = smf.mixedlm(formula="indiff_point ~ C(Group_Cocaine) + C(Commodity_Health) + C(Commodity_Money) + C(Gain_Loss_Gains) + C(Delay_Prob_Delay) + Amount + IV", 
                      data=trim_df, groups='pid')
  regr.fit()
  observed_data = trim_df['indiff_point']
  predictions = regr.predict(regr.fit().fe_params, trim_df[preds])
  r2 = metrics.r2_score(observed_data, predictions)
  mse_ = metrics.mean_squared_error(observed_data, predictions)
  rmse_ = np.sqrt(mse_)
  mae_ = metrics.mean_absolute_error(observed_data, predictions)
  num_params = len(regr.exog_names)
  try:
    aic_ = calc_aic(n=len(observed_data), loss=mse_, num_params=num_params)
  except:
    aic_ = np.nan
  try:
    bic_ = calc_bic(n=len(observed_data), loss=mse_, num_params=num_params)
  except:
    bic_ = np.nan
  
  # Save values
  fit.append('all_ME')
  vac.append(r2)
  mae.append(mae_)
  mse.append(mse_)
  rmse.append(rmse_)
  aic.append(aic_)
  bic.append(bic_)
  models.append(model)
  group.append('all')

  # Update notebook user
  clear_output()
  print(f'{i+1}% complete...')

  # Final save
fit_df = pd.DataFrame({'fit':fit, 'vac':vac, 'mse':mse, 'rmse':rmse, 
                       'mae':mae, 'aic':aic, 'bic':bic, 'model': models, 'group':group})
fit_df.to_csv(f'../Data/02_initial_models_output/boostrapped_ml_regressors_{model}.csv')
fit_df

99% complete...


,fit,vac,mse,rmse,mae,aic,bic,model,group
0,all_ME,-0.499686,0.236793,0.486614,0.410545,-6740.261314,-6688.635811,ME,all
1,all_ME,-0.593143,0.252764,0.502757,0.421839,-6434.143960,-6382.518457,ME,all
2,all_ME,-0.540190,0.240189,0.490090,0.412976,-6673.492723,-6621.867220,ME,all
3,all_ME,-0.548458,0.244560,0.494531,0.419811,-6588.892710,-6537.267207,ME,all
4,all_ME,-0.545097,0.244911,0.494884,0.417297,-6582.182262,-6530.556759,ME,all
...,...,...,...,...,...,...,...,...,...
95,all_ME,-0.496320,0.237223,0.487056,0.410174,-6731.752213,-6680.126710,ME,all
96,all_ME,-0.582604,0.248669,0.498668,0.423351,-6510.747608,-6459.122105,ME,all
97,all_ME,-0.565716,0.246197,0.496183,0.419887,-6557.608446,-6505.982943,ME,all
98,all_ME,-0.586435,0.253578,0.503565,0.423132,-6419.081958,-6367.456455,ME,all


In [ ]:
# Using bootstrapped samples to fit MLM to Cocaine group only
model = 'ME'
import warnings
warnings.filterwarnings("ignore")
fit = []
vac = []
mse =[]
mae = []
rmse =[]
aic = []
bic = []
models = []
group = []

# Predictor cols
preds = ['Commodity_Cocaine', 'Commodity_Health', 'Commodity_Money', 
         'Gain_Loss_Gains', 'Delay_Prob_Delay', 'Amount', 'IV']

# Isolate just individuals from the cocaine group
trim_df = encode_df[encode_df['Group_Cocaine']==1]

for i in range(0, 100):
  rand_state = int(round(random.random()*10000, 0))
  trim_df = trim_df.sample(n=int(round((len(encode_df)*0.8), 2)), replace=True, random_state=rand_state)
  trim_df = trim_df.reset_index(drop=True)
  # Fit model
  regr = smf.mixedlm(formula="indiff_point ~ C(Commodity_Health) + C(Commodity_Money) + C(Gain_Loss_Gains) + C(Delay_Prob_Delay) + Amount + IV", 
                      data=trim_df, groups='pid')
  regr.fit()
  observed_data = trim_df['indiff_point']
  predictions = regr.predict(regr.fit().fe_params, trim_df[preds])
  r2 = metrics.r2_score(observed_data, predictions)
  mse_ = metrics.mean_squared_error(observed_data, predictions)
  rmse_ = np.sqrt(mse_)
  mae_ = metrics.mean_absolute_error(observed_data, predictions)
  num_params = len(regr.exog_names)
  try:
    aic_ = calc_aic(n=len(observed_data), loss=mse_, num_params=num_params)
  except:
    aic_ = np.nan
  try:
    bic_ = calc_bic(n=len(observed_data), loss=mse_, num_params=num_params)
  except:
    bic_ = np.nan
  
  # Save values
  fit.append('cocaine_ME')
  vac.append(r2)
  mae.append(mae_)
  mse.append(mse_)
  rmse.append(rmse_)
  aic.append(aic_)
  bic.append(bic_)
  models.append(model)
  group.append('cocaine')

  # Update notebook user
  clear_output()
  print(f'{i+1}% complete...')

  # Final save
fit_df_2 = pd.DataFrame({'fit':fit, 'vac':vac, 'mse':mse, 'rmse':rmse, 
                       'mae':mae, 'aic':aic, 'bic':bic, 'model': models, 'group':group})
fit_df = fit_df.append(fit_df_2, ignore_index=True)
fit_df.to_csv(f'../Data/02_initial_models_output/boostrapped_ml_regressors_{model}.csv')
fit_df

99% complete...


,fit,vac,mse,rmse,mae,aic,bic,model,group
0,all_ME,-0.499686,0.236793,0.486614,0.410545,-6740.261314,-6688.635811,ME,all
1,all_ME,-0.593143,0.252764,0.502757,0.421839,-6434.143960,-6382.518457,ME,all
2,all_ME,-0.540190,0.240189,0.490090,0.412976,-6673.492723,-6621.867220,ME,all
3,all_ME,-0.548458,0.244560,0.494531,0.419811,-6588.892710,-6537.267207,ME,all
4,all_ME,-0.545097,0.244911,0.494884,0.417297,-6582.182262,-6530.556759,ME,all
...,...,...,...,...,...,...,...,...,...
195,cocaine_ME,-3.107174,0.629942,0.793689,0.706517,-2153.377040,-2108.204725,ME,cocaine
196,cocaine_ME,-3.278720,0.645765,0.803595,0.716484,-2037.029858,-1991.857543,ME,cocaine
197,cocaine_ME,-2.855569,0.580627,0.761989,0.675172,-2535.706710,-2490.534395,ME,cocaine
198,cocaine_ME,-2.780117,0.562860,0.750240,0.664139,-2681.453312,-2636.280997,ME,cocaine


In [ ]:
# Using bootstrapped samples to fit MLM to Control group only
model = 'ME'
import warnings
warnings.filterwarnings("ignore")
fit = []
vac = []
mse =[]
mae = []
rmse =[]
aic = []
bic = []
models = []
group = []

# Predictor cols
preds = ['Commodity_Cocaine', 'Commodity_Health', 'Commodity_Money', 
         'Gain_Loss_Gains', 'Delay_Prob_Delay', 'Amount', 'IV']

# Isolate just individuals from the cocaine group
trim_df = encode_df[encode_df['Group_Cocaine']==0]

for i in range(0, 100):
  rand_state = int(round(random.random()*10000, 0))
  trim_df = trim_df.sample(n=int(round((len(encode_df)*0.8), 2)), replace=True, random_state=rand_state)
  trim_df = trim_df.reset_index(drop=True)
  # Fit model
  regr = smf.mixedlm(formula="indiff_point ~ C(Commodity_Health) + C(Commodity_Money) + C(Gain_Loss_Gains) + C(Delay_Prob_Delay) + Amount + IV", 
                      data=trim_df, groups='pid')
  try:
    regr.fit()
  except:
    continue
  observed_data = trim_df['indiff_point']
  predictions = regr.predict(regr.fit().fe_params, trim_df[preds])
  r2 = metrics.r2_score(observed_data, predictions)
  mse_ = metrics.mean_squared_error(observed_data, predictions)
  rmse_ = np.sqrt(mse_)
  mae_ = metrics.mean_absolute_error(observed_data, predictions)
  num_params = len(regr.exog_names)
  try:
    aic_ = calc_aic(n=len(observed_data), loss=mse_, num_params=num_params)
  except:
    aic_ = np.nan
  try:
    bic_ = calc_bic(n=len(observed_data), loss=mse_, num_params=num_params)
  except:
    bic_ = np.nan
  
  # Save values
  fit.append('control_ME')
  vac.append(r2)
  mae.append(mae_)
  mse.append(mse_)
  rmse.append(rmse_)
  aic.append(aic_)
  bic.append(bic_)
  models.append(model)
  group.append('control')

  # Update notebook user
  clear_output()
  print(f'{i+1}% complete...')

  # Final save
fit_df_2 = pd.DataFrame({'fit':fit, 'vac':vac, 'mse':mse, 'rmse':rmse, 
                       'mae':mae, 'aic':aic, 'bic':bic, 'model': models, 'group':group})
fit_df = fit_df.append(fit_df_2, ignore_index=True)
fit_df.to_csv(f'../Data/02_initial_models_output/boostrapped_ml_regressors_{model}.csv')
fit_df

99% complete...


,fit,vac,mse,rmse,mae,aic,bic,model,group
0,all_ME,-0.499686,0.236793,0.486614,0.410545,-6740.261314,-6688.635811,ME,all
1,all_ME,-0.593143,0.252764,0.502757,0.421839,-6434.143960,-6382.518457,ME,all
2,all_ME,-0.540190,0.240189,0.490090,0.412976,-6673.492723,-6621.867220,ME,all
3,all_ME,-0.548458,0.244560,0.494531,0.419811,-6588.892710,-6537.267207,ME,all
4,all_ME,-0.545097,0.244911,0.494884,0.417297,-6582.182262,-6530.556759,ME,all
...,...,...,...,...,...,...,...,...,...
240,control_ME,-3733.948574,543.113926,23.304805,23.301954,29548.426607,29593.598922,ME,control
241,control_ME,-15.210572,2.300424,1.516715,1.473447,3921.209217,3966.381532,ME,control
242,control_ME,-52.345573,7.691120,2.773287,2.749776,9581.911249,9627.083564,ME,control
243,control_ME,-401.005789,57.035891,7.552211,7.543892,18978.862622,19024.034937,ME,control
